<a href="https://colab.research.google.com/github/nyee88/Brilliant/blob/main/Brilliant20250819.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Support for third party widgets will remain active for the duration of the session. To disable support:

In [10]:
from IPython.display import HTML, display

display(HTML("""
<div style="font-family:system-ui,Segoe UI,Roboto,Arial,sans-serif; max-width:980px;">
  <div id="wrap" style="position:relative; width:900px; height:280px; user-select:none;">
    <canvas id="scene" style="position:absolute; inset:0; width:100%; height:100%;"></canvas>
    <canvas id="ball"  style="position:absolute; inset:0; width:100%; height:100%; pointer-events:none;"></canvas>
    <button id="go" style="position:absolute; right:12px; top:12px; padding:6px 12px; border:1px solid #bbb; border-radius:8px; background:#f3f3f3; font-weight:600; cursor:pointer;">GO</button>
    <div id="msg" style="position:absolute; right:12px; top:50px; color:#d22; font-weight:600; opacity:0; transition:opacity .25s;">Not conjugated — charge cannot travel</div>
  </div>
</div>

<script>
(() => {
  const DPR = window.devicePixelRatio || 1;
  const scene = document.getElementById('scene');
  const ball  = document.getElementById('ball');
  const sctx  = scene.getContext('2d');
  const bctx  = ball.getContext('2d');

  function fit(cvs){
    const r = cvs.getBoundingClientRect();
    cvs.width  = Math.floor(r.width  * DPR);
    cvs.height = Math.floor(r.height * DPR);
    cvs.getContext('2d').setTransform(DPR,0,0,DPR,0,0);
    return {w:r.width, h:r.height};
  }
  let S = fit(scene); fit(ball);

  // ---- geometry ----
  const N = 8;
  const marginL = 60, marginR = 60;
  const stepX = (S.w - marginL - marginR)/(N-1);
  const baseY = 160, zig = 35;
  const coords = Array.from({length:N}, (_,i)=>({x: marginL + i*stepX, y: baseY + (i%2? -zig:0)}));
  const bonds  = Array.from({length:N-1}, (_,i)=>[i,i+1]);
  const clickable = new Set([0,2,4,6]);     // alternating
  const bondState = Array(bonds.length).fill(0); // 0 single, 1 double

  // ---- helpers ----
  function isConjugatedFull(){
    let evenD=true, oddS=true, oddD=true, evenS=true;
    for (let i=0;i<bondState.length;i++){
      if (i%2===0){ if(bondState[i]!==1) evenD=false; if(bondState[i]!==0) evenS=false; }
      else        { if(bondState[i]!==0) oddS=false;  if(bondState[i]!==1) oddD=false; }
    }
    return (evenD && oddS) || (oddD && evenS);
  }
  const anyDouble = () => bondState.some(v=>v===1);
  function activeAtomIndices(){
    const s = new Set();
    for (let i=0;i<bonds.length;i++) if (bondState[i]===1){ s.add(bonds[i][0]); s.add(bonds[i][1]); }
    return Array.from(s).sort((a,b)=>a-b);
  }

  // ---- draw (atoms/bonds) ----
  const ATOM_R=7, SINGLE_W=2, DOUBLE_SEP=6;

  function drawBonds(){
    sctx.lineWidth = SINGLE_W;
    sctx.strokeStyle = '#000';
    for (let i=0;i<bonds.length;i++){
      const [a,b] = bonds[i], p1=coords[a], p2=coords[b];
      if (bondState[i]===0){
        sctx.beginPath(); sctx.moveTo(p1.x,p1.y); sctx.lineTo(p2.x,p2.y); sctx.stroke();
      } else {
        const vx=p2.x-p1.x, vy=p2.y-p1.y, L=Math.hypot(vx,vy)||1;
        const nx=-vy/L, ny=vx/L, dx=nx*(DOUBLE_SEP/2), dy=ny*(DOUBLE_SEP/2);
        sctx.beginPath(); sctx.moveTo(p1.x+dx,p1.y+dy); sctx.lineTo(p2.x+dx,p2.y+dy); sctx.stroke();
        sctx.beginPath(); sctx.moveTo(p1.x-dx,p1.y-dy); sctx.lineTo(p2.x-dx,p2.y-dy); sctx.stroke();
      }
    }
  }
  function drawAtoms(){
    sctx.lineWidth = 1.5;
    sctx.strokeStyle = '#1f77b4';
    sctx.fillStyle   = '#cfe8ff';
    for (const p of coords){ sctx.beginPath(); sctx.arc(p.x,p.y,ATOM_R,0,Math.PI*2); sctx.fill(); sctx.stroke(); }
  }

  // ---- pz orbitals: 10× scale (with clamping so they stay on-canvas) ----
  function drawOrbitals(){
    if (!anyDouble()) return;
    const act = activeAtomIndices(); if (!act.length) return;

    sctx.save();
    sctx.globalCompositeOperation = 'source-over';
    sctx.filter = 'blur(6px)';   // soft, not smeary

    const SCALE = 2.5            // <<—— main scale knob
    const alphaMax = 0.60;
    const tint = '70,140,230';   // RGB for blue

    // size from spacing; clamp to viewport so huge lobes don't blow past edges
    const maxHoriz = S.w * 0.45;
    const maxHeight = S.h * 0.45;
    const baseHoriz = stepX * 0.62;
    const horiz = Math.min(baseHoriz * SCALE, maxHoriz);  // half-width
    const height = Math.min(38 * SCALE, maxHeight);       // lobe height

    function lobe(cx, cy, sign){
      const tipY  = cy - sign*height;
      const midY  = cy - sign*height*0.52;
      const wNode = Math.max(3, horiz*0.05);   // tight waist at node
      const wMax  = horiz;                     // full width for overlap
      sctx.beginPath();
      sctx.moveTo(cx, cy);
      sctx.bezierCurveTo(cx - wNode, cy - sign*10,
                         cx - wMax,  midY,
                         cx,         tipY);
      sctx.bezierCurveTo(cx + wMax,  midY,
                         cx + wNode, cy - sign*10,
                         cx,         cy);
      const g = sctx.createLinearGradient(cx, cy, cx, tipY);
      g.addColorStop(0.00, `rgba(${tint},0.00)`);                  // node = 0
      g.addColorStop(0.45, `rgba(${tint},${alphaMax*0.65})`);
      g.addColorStop(1.00, `rgba(${tint},${alphaMax})`);
      sctx.fillStyle = g;
      sctx.fill();
    }

    for (const i of act){
      const p = coords[i];
      lobe(p.x, p.y, -1);  // upper
      lobe(p.x, p.y, +1);  // lower
    }
    sctx.restore();
  }

  function drawTitleIfConjugated(){
    if (!isConjugatedFull()) return;
    sctx.fillStyle = 'green';
    sctx.font = '600 14px system-ui,Segoe UI,Roboto,Arial';
    sctx.fillText('Conjugated — charge can travel', 20, 24);
  }

  function drawScene(){
    sctx.clearRect(0,0,S.w,S.h);
    drawOrbitals();      // underlay
    drawBonds();
    drawAtoms();
    drawTitleIfConjugated();
  }

  // ---- clicks ----
  function distPointSeg(px,py, ax,ay, bx,by){
    const vx=bx-ax, vy=by-ay, wx=px-ax, wy=py-ay, L2=vx*vx+vy*vy;
    if(L2===0) return Math.hypot(wx,wy);
    let t=(wx*vx+wy*vy)/L2; t=Math.max(0,Math.min(1,t));
    const cx=ax+t*vx, cy=ay+t*vy;
    return Math.hypot(px-cx, py-cy);
  }
  scene.addEventListener('click', (ev)=>{
    const r = scene.getBoundingClientRect();
    const px = ev.clientX - r.left, py = ev.clientY - r.top;
    let best=-1, bestD=Infinity;
    for (let i=0;i<bonds.length;i++){
      if (!clickable.has(i)) continue;
      const [a,b]=bonds[i], A=coords[a], B=coords[b];
      const d = distPointSeg(px,py, A.x,A.y, B.x,B.y);
      if (d<bestD){ bestD=d; best=i; }
    }
    if (best>=0 && bestD<=12){
      bondState[best] ^= 1;
      drawScene(); drawBall(Apx);
    }
  });

  // ---- ball animation (GO only when fully conjugated) ----
  const goBtn = document.getElementById('go');
  const msg   = document.getElementById('msg');
  const trackY = baseY - 55;
  const Apx = {x: coords[0].x - 40, y: trackY};
  const Bpx = {x: coords[coords.length-1].x + 40, y: trackY};

  function drawBall(p){
    bctx.clearRect(0,0,S.w,S.h);
    bctx.beginPath(); bctx.arc(p.x,p.y,10,0,Math.PI*2);
    bctx.fillStyle='#FFA500'; bctx.strokeStyle='#CC7000'; bctx.lineWidth=2;
    bctx.fill(); bctx.stroke();
  }
  drawBall(Apx);

  const DUR=1500, ease=t=>t*t*(3-2*t);
  let running=false, t0=null, raf=null;
  function step(ts){
    if(!t0) t0=ts;
    let u=(ts-t0)/DUR; if(u>=1){u=1; running=false;}
    u=ease(Math.max(0,Math.min(1,u)));
    drawBall({x: Apx.x + (Bpx.x - Apx.x)*u, y:Apx.y});
    if(running) raf=requestAnimationFrame(step);
  }
  function shake(el){ el.animate([{transform:'translateX(0)'},{transform:'translateX(-6px)'},{transform:'translateX(6px)'},{transform:'translateX(0)'}],{duration:250}); }
  goBtn.addEventListener('click', ()=>{
    if (!isConjugatedFull()){ msg.style.opacity=1; setTimeout(()=>msg.style.opacity=0,900); shake(goBtn); return; }
    if(running) cancelAnimationFrame(raf);
    running=true; t0=null; requestAnimationFrame(step);
  });

  drawScene();
  window.addEventListener('resize', ()=>{ S=fit(scene); fit(ball); drawScene(); drawBall(Apx); });
})();
</script>
"""))
